In [12]:
import os

In [13]:
%pwd

'a:\\Chicken Project\\classification_of_chicken_Disease\\research'

In [14]:
os.chdir("../")

In [15]:
%pwd

'a:\\Chicken Project\\classification_of_chicken_Disease'

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [19]:
import time

In [20]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [21]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [7]:
print(tf.__version__)

2.18.0


In [14]:
tf.compat.v1.enable_eager_execution()

[2025-01-24 14:11:25,157: WARNING: module_wrapper: From C:\Users\pawar\AppData\Local\Temp\ipykernel_776\1112888837.py:1: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
]


In [8]:
import tensorflow as tf

# Enable eager execution (if using TensorFlow 1.x compatibility mode)
tf.compat.v1.enable_eager_execution()

# Check if eager execution is enabled
print("Eager execution enabled:", tf.executing_eagerly())

[2025-01-24 21:02:08,828: WARNING: module_wrapper: From C:\Users\pawar\AppData\Local\Temp\ipykernel_28408\4052337522.py:4: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
]
Eager execution enabled: True


In [19]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [22]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [23]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-01-24 21:02:55,723: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-24 21:02:55,723: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-24 21:02:55,723: INFO: common: created directory at: artifacts]
[2025-01-24 21:02:55,731: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-01-24 21:02:55,732: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-01-24 21:02:55,732: INFO: common: created directory at: artifacts\training]
[2025-01-24 21:02:56,142: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6455 - loss: 0.8165

a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[2025-01-24 21:03:43,720: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.6506 - loss: 0.8053 - val_accuracy: 0.8750 - val_loss: 0.4572
[2025-01-24 21:03:43,879: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


In [21]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [22]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
print("Eager execution enabled:", tf.executing_eagerly())


Eager execution enabled: True


In [23]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        return history


In [24]:
@tf.function
def train_model(model, train_generator, valid_generator, steps_per_epoch, validation_steps, epochs, callbacks):
    return model.fit(
        train_generator,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        validation_data=valid_generator,
        callbacks=callbacks
    )

# In your main execution:
history = train_model(
    training.model,
    training.train_generator,
    training.valid_generator,
    training.steps_per_epoch,
    training.validation_steps,
    training.config.params_epochs,
    callback_list
)


a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\callbacks\tensorboard.py:680: UserWarning: Model failed to serialize as JSON. Ignoring... numpy() is only available when eager execution is enabled.
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")


NotImplementedError: in user code:

    File "C:\Users\pawar\AppData\Local\Temp\ipykernel_776\4253769890.py", line 9, in train_model  *
        callbacks=callbacks
    File "a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\backend\tensorflow\core.py", line 155, in convert_to_numpy
        return np.array(x)

    NotImplementedError: numpy() is only available when eager execution is enabled.


In [25]:
def create_dataset(generator, batch_size):
    return tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None, *generator.image_shape], [None, generator.num_classes])
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# In your Training class:
def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        self.train_dataset = create_dataset(self.train_generator, self.config.params_batch_size)
        self.valid_dataset = create_dataset(self.valid_generator, self.config.params_batch_size)

# Update the train method to use these datasets
def train(self, callback_list: list):
    history = self.model.fit(
        self.train_dataset,
        epochs=self.config.params_epochs,
        validation_data=self.valid_dataset,
        callbacks=callback_list
    )
    self.save_model(
        path=self.config.trained_model_path,
        model=self.model
    )
    return history


In [26]:
@tf.function
def train_model(model, train_generator, valid_generator, steps_per_epoch, validation_steps, epochs, callbacks):
    return model.fit(
        train_generator,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        validation_data=valid_generator,
        callbacks=callbacks
    )

# In your main execution:
history = train_model(
    training.model,
    training.train_generator,
    training.valid_generator,
    training.steps_per_epoch,
    training.validation_steps,
    training.config.params_epochs,
    callback_list
)


NotImplementedError: in user code:

    File "C:\Users\pawar\AppData\Local\Temp\ipykernel_776\4253769890.py", line 9, in train_model  *
        callbacks=callbacks
    File "a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "a:\Chicken Project\classification_of_chicken_Disease\venv\lib\site-packages\keras\src\backend\tensorflow\core.py", line 155, in convert_to_numpy
        return np.array(x)

    NotImplementedError: numpy() is only available when eager execution is enabled.
